# Notebook de la dimensión Cliente - Proceso ETL

## 1. Configuración de conexiones (fuente y bodega)

In [39]:
import yaml
from sqlalchemy import create_engine
import pandas as pd

# Leer configuración desde archivo externo
with open('config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_fuente = config['fuente']
    config_bodega = config['bodega']

# URLs de conexión
url_fuente = (f"{config_fuente['driver']}://{config_fuente['user']}:{config_fuente['password']}@"
              f"{config_fuente['host']}:{config_fuente['port']}/{config_fuente['db']}")

url_bodega = (f"{config_bodega['driver']}://{config_bodega['user']}:{config_bodega['password']}@"
              f"{config_bodega['host']}:{config_bodega['port']}/{config_bodega['db']}")

# Engines SQLAlchemy
engine_fuente = create_engine(url_fuente)
engine_bodega = create_engine(url_bodega)

## 2. Extracción de datos desde la base de datos - Extract

In [40]:
from sqlalchemy import inspect

# Saber las columnas que hay en la base de datos OLTP
inspector = inspect(engine_fuente)
tablas = inspector.get_table_names()
print("Tablas de la base de datos:")
print(tablas)

Tablas de la base de datos:
['admin_interface_theme', 'areas_cliente', 'auth_user_user_permissions', 'clientes_coordinador', 'clientes_datosmensajero', 'clientes_mensajeroaquitoy_clientes', 'auth_permission', 'auth_group', 'auth_user_groups', 'auth_user', 'authtoken_token', 'ciudad', 'cliente', 'clientes_mensajeroaquitoy', 'area', 'django_migrations', 'departamento', 'clientes_usuarioaquitoy', 'django_admin_log', 'django_session', 'mensajeria_novedadesservicio', 'mensajeria_tiponovedad', 'mensajeria_origenservicio', 'mensajeria_tipopago', 'mensajeria_estado', 'mensajeria_tiposervicio', 'mensajeria_servicio', 'mensajeria_estadosservicio', 'mensajeria_tipovehiculo', 'sede', 'auth_group_permissions', 'django_content_type', 'tipo_cliente', 'mensajeria_destinoservicio', 'mensajeria_documentoasociado']


In [41]:
# Extracción de datos desde las tablas cliente y tipo_cliente
df_cliente_raw = pd.read_sql("SELECT * FROM cliente;", engine_fuente)
df_tipo_cliente = pd.read_sql("SELECT * FROM tipo_cliente;", engine_fuente)

# Vista
df_cliente_raw.head()

,cliente_id,nit_cliente,nombre,email,direccion,telefono,nombre_contacto,ciudad_id,tipo_cliente_id,activo,coordinador_id,sector
0,1,25,Cliente 2,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,1,1,True,NaN,S
1,2,123,Cliente 1,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,1,1,True,2.0,industrial
2,6,24390-3,CLINICA DEPORTIVA DEL SUR,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,1,1,True,1.0,salud
3,19,8301821,HOSPITAL ORTOPEDICO DE COLOMBIA,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,1,1,True,NaN,salud
4,8,5017350-8,CLINICA NEFROLOGOS DE CALI,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,1,1,True,NaN,salud


## 3. Transformación de los datos - Transform

In [42]:
# Unión de cliente con tipo_cliente
df_cliente = df_cliente_raw.merge(
    df_tipo_cliente,
    left_on='tipo_cliente_id',
    right_on='tipo_cliente_id',
    how='left',
    suffixes=('', '_tipo')  # para evitar colisión de nombres
)

In [43]:
# Selección y renombre de columnas
df_dim_cliente = df_cliente[[
    'cliente_id',
    'nombre',
    'nombre_tipo'
]].rename(columns={
    'cliente_id': 'id_cliente',
    'nombre': 'Nombre_Cliente',
    'nombre_tipo': 'Tipo_Cliente'
})

In [44]:
# Verificar
print(df_dim_cliente.head())

   id_cliente                   Nombre_Cliente      Tipo_Cliente
0           1                        Cliente 2  Persona Juridica
1           2                        Cliente 1  Persona Juridica
2           6       CLINICA  DEPORTIVA DEL SUR  Persona Juridica
3          19  HOSPITAL ORTOPEDICO DE COLOMBIA  Persona Juridica
4           8      CLINICA NEFROLOGOS  DE CALI  Persona Juridica


In [45]:
from datetime import datetime

# Agregar columna de fecha de carga
df_dim_cliente["fecha_carga"] = datetime.now().date()

In [46]:
# Eliminar duplicados
df_dim_cliente = df_dim_cliente.drop_duplicates(subset=['id_cliente'])

In [47]:
# Verificar resultado
df_dim_cliente.head()

,id_cliente,Nombre_Cliente,Tipo_Cliente,fecha_carga
0,1,Cliente 2,Persona Juridica,2025-06-05
1,2,Cliente 1,Persona Juridica,2025-06-05
2,6,CLINICA DEPORTIVA DEL SUR,Persona Juridica,2025-06-05
3,19,HOSPITAL ORTOPEDICO DE COLOMBIA,Persona Juridica,2025-06-05
4,8,CLINICA NEFROLOGOS DE CALI,Persona Juridica,2025-06-05


## 4. Guardado de la dimensión - Load

In [48]:
# Guardar como archivo CSV
df_dim_cliente.to_csv("dim_cliente.csv", index=False)

In [49]:
# Cargar en la bodega de datos OLAP
df_dim_cliente.to_sql("dim_cliente", engine_bodega, if_exists="replace", index=False)

27

In [50]:
# Verificación del archivo
df_verificacion = pd.read_csv("dim_cliente.csv")
print(df_verificacion.head())

   id_cliente                   Nombre_Cliente      Tipo_Cliente fecha_carga
0           1                        Cliente 2  Persona Juridica  2025-06-05
1           2                        Cliente 1  Persona Juridica  2025-06-05
2           6       CLINICA  DEPORTIVA DEL SUR  Persona Juridica  2025-06-05
3          19  HOSPITAL ORTOPEDICO DE COLOMBIA  Persona Juridica  2025-06-05
4           8      CLINICA NEFROLOGOS  DE CALI  Persona Juridica  2025-06-05
